# 0. Install dependencies and libraries

In [13]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score
import mediapipe as mp
import pandas as pd
import numpy as np
import cv2
import pickle
import time


# Disable jedi autocompleter
%config Completer.use_jedi = False

In [14]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

   ## 4. Make detections with Model

In [15]:
# Reload the saved model
with open('body_language.pkl', 'rb')  as f:
    model = pickle.load(f)

In [16]:
model

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestclassifier', RandomForestClassifier())])

In [17]:
cap = cv2.VideoCapture(0) # number varies with cameras

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor the image to RGB from BGR, as mediapipe expects it
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        
        
        # Make detections
        image_rgb.flags.writeable = False  # for perofmance enhancement for holistic model
        results = holistic.process(image_rgb)
        #print(results.face_landmarks)
        
        # Face_landmarks, pose_landmarks, left_hand_landmarks and right_hand_landmark       
        # Recolor back to BGR for rendering image in openCV
        image_rgb.flags.writeable = True  # default  perofmance tuner back for openCV
        image_bgr = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)
        
        # 1. Mark face landamarks, 
        mp_drawing.draw_landmarks(image_bgr,
                                  results.face_landmarks,
                                  mp_holistic.FACE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(80,110,10), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(80,256,121), thickness=2, circle_radius=2)
                                 )
        
        # 2. Mark Right Hand landmarks
        mp_drawing.draw_landmarks(image_bgr, 
                                  results.right_hand_landmarks, 
                                  mp_holistic.HAND_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(120,22,10), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )
        
        # 3. Mark Left Hand landmarks
        mp_drawing.draw_landmarks(image_bgr, 
                                  results.left_hand_landmarks, 
                                  mp_holistic.HAND_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(190,22,76), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )
        
        # 4. Mark Pose Landmarks
        mp_drawing.draw_landmarks(image_bgr, 
                                  results.pose_landmarks, 
                                  mp_holistic.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(245,66,23), thickness=2, circle_radius=4)
                                 )
        
        # Export face and pose co-oridnates to CSV
        try:
            # Extract pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(
                np.array(
                [[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]
            ).flatten()
            )
            
            # Extract face landmarks
            
            face = results.face_landmarks.landmark
            face_row = list(
                np.array(
                [[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]
            ).flatten()
            )  ### Note: there's no visibility co-oridnate in face_landmarks, its always be zero
            
            # Concate rows
            row = pose_row + face_row
            
            X = pd.DataFrame([row])
            body_language_class = model.predict(X)[0]
            body_language_probability = model.predict_proba(X)[0]  ## probability of correct classification
            
            # print 
            #print(body_language_class, body_language_probability)
            
            # Grab Ear co-ordinates
            coords = tuple(
                np.multiply(
                    np.array(
                        (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x,
                         results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y)
                    ),[640, 480]
                )
                .astype(int))
            
            cv2.rectangle(image_bgr,
                          (coords[0], coords[1]+5),
                          (coords[0]+len(body_language_class)*10,coords[1]-30),
                          (245,117, 16), -1)
            cv2.putText(image_bgr, 
                        body_language_class, 
                        coords,
                        cv2.FONT_HERSHEY_SIMPLEX, 
                        1,
                        (255, 255, 255),
                        2,
                        cv2.LINE_AA
                       )
            # Add Status Box
            cv2.rectangle(image_bgr,(0,0),(250,60),(245,117, 16), -1)
            
            # Display CLASS
            cv2.putText(image_bgr,'CLASS',(95,12),cv2.FONT_HERSHEY_SIMPLEX, 0.5,(0, 0, 0),1,cv2.LINE_AA)
            cv2.putText(image_bgr,body_language_class.split(' ')[0],
                        (90,40),cv2.FONT_HERSHEY_SIMPLEX, 1,(255, 255, 255),2,cv2.LINE_AA)


            
            # Display Prediction Probability
            cv2.putText(image_bgr,'PROB',(15,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0, 0, 0),1,cv2.LINE_AA)
            cv2.putText(image_bgr,str(round(body_language_probability[np.argmax(body_language_probability)],2)),
                        (10,40),cv2.FONT_HERSHEY_SIMPLEX, 1,(255, 255, 255),2,cv2.LINE_AA)
    
        except:
            pass

        # Render back image frame onto the web cam screen
        cv2.imshow('Webcam video for Holistic model', image_bgr)

        if cv2.waitKey(10) & 0xFF==ord('q'):
            break
        
cap.release()

cv2.destroyAllWindows()